# 01 Attribute Network and Make Scenario Pickle

In [1]:
import os
import sys
import yaml

import pandas as pd

from network_wrangler import RoadwayNetwork
from network_wrangler import TransitNetwork
from network_wrangler import ProjectCard
from network_wrangler import Scenario
from network_wrangler import WranglerLogger

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters
from lasso import mtc

import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import logging
logger = logging.getLogger("WranglerLogger")
logger.handlers[0].stream = sys.stdout
# if you don't want to see so much detail, set to logging.INFO or DEBUG
logger.setLevel(logging.INFO)

## Remote I/O

In [4]:
root_dir = "C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy"
input_dir = os.path.join(root_dir, 'data', 'processed', 'version_03')
output_dir = input_dir
lasso_dir = "C:/Users/USGS667806/Lasso"

In [5]:
parameters = Parameters(lasso_base_dir = lasso_dir)

2021-02-10 17:11:31, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:11:31, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:11:31, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:11:31, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso


## Data Reads

In [6]:
link_file = os.path.join(input_dir, 'link.json')
node_file = os.path.join(input_dir, 'node.geojson')
shape_file = os.path.join(input_dir, 'shape.geojson')

roadway_net = RoadwayNetwork.read(
    link_filename = link_file, 
    node_filename = node_file, 
    shape_filename = shape_file
)

2021-02-10 17:11:31, INFO: Reading RoadwayNetwork
2021-02-10 17:11:31, INFO: Reading RoadwayNetwork
2021-02-10 17:14:01, INFO: Read 1634769 links from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_03\link.json
2021-02-10 17:14:01, INFO: Read 1634769 links from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_03\link.json
2021-02-10 17:14:01, INFO: Read 644475 nodes from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_03\node.geojson
2021-02-10 17:14:01, INFO: Read 644475 nodes from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_03\node.geojson
2021-02-10 17:14:01, INFO: Read 869970 shapes from C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_03\shape.geojson
2021-02-10 17:14:01, INFO: Read 869970 shapes from C:/Users/USGS667806/Documents/GitHub/travel-model-two-ne

*Please note*: a third party library is used to read in the GTFS feed and it will generate warnings such as, `INFO: Removing calendar.txt from transit network config because file not found`. These warnings can be ignored, as the standard Network Wrangler transit network does not need these files. 

In [7]:
transit_net = TransitNetwork.read(feed_path = input_dir)

2021-02-10 17:18:19, INFO: Read in transit feed from: C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_03
2021-02-10 17:18:19, INFO: Read in transit feed from: C:/Users/USGS667806/Documents/GitHub/travel-model-two-networks - Copy\data\processed\version_03
2021-02-10 17:18:19, INFO: Removing calendar.txt from transit network config because file not found
2021-02-10 17:18:19, INFO: Removing calendar.txt from transit network config because file not found
2021-02-10 17:18:19, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-02-10 17:18:19, INFO: Removing calendar_dates.txt from transit network config because file not found
2021-02-10 17:18:20, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-02-10 17:18:20, INFO: Removing fare_attributes.txt from transit network config because file not found
2021-02-10 17:18:20, INFO: Removing fare_rules.txt from transit network co

## Attribute the Network

In [8]:
r_net = mtc.determine_number_of_lanes(
    roadway_network = roadway_net,
    parameters = parameters,
    network_variable = 'lanes'
)

r_net.links_df.lanes.value_counts()

2021-02-10 17:18:22, INFO: Determining number of lanes
2021-02-10 17:18:22, INFO: Determining number of lanes
2021-02-10 17:18:22, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:18:22, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:18:22, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:18:22, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:22:10, INFO: Finished determining number of lanes using variable: lanes
2021-02-10 17:22:10, INFO: Finished determining number of lanes using variable: lanes


1    1486397
2     104207
3      22455
4      14124
5       6504
6        929
7        134
8         18
9          1
Name: lanes, dtype: int64

In [9]:
r_net = mtc.calculate_facility_type(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'ft')

r_net.links_df.ft.value_counts()

2021-02-10 17:22:10, INFO: Calculating Facility Type
2021-02-10 17:22:10, INFO: Calculating Facility Type
2021-02-10 17:22:10, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:22:10, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:22:10, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:22:10, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:06, INFO: Finished calculating roadway class variable: ft
2021-02-10 17:24:06, INFO: Finished calculating roadway class variable: ft


99    890153
7     538666
6      96883
5      58344
4      38332
3       6365
2       3252
1       2774
Name: ft, dtype: int64

In [10]:
r_net = mtc.calculate_useclass(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'useclass')

r_net.links_df.useclass.value_counts()

2021-02-10 17:24:07, INFO: Determining useclass
2021-02-10 17:24:07, INFO: Determining useclass
2021-02-10 17:24:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:07, INFO: Calculating and adding roadway network variable: useclass
2021-02-10 17:24:07, INFO: Calculating and adding roadway network variable: useclass
2021-02-10 17:24:07, INFO: Finished determining variable: useclass
2021-02-10 17:24:07, INFO: Finished determining variable: useclass


0    1634769
Name: useclass, dtype: int64

In [11]:
r_net = mtc.calculate_assignable(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'assignable')

r_net.links_df.assignable.value_counts()

2021-02-10 17:24:07, INFO: Determining assignable
2021-02-10 17:24:07, INFO: Determining assignable
2021-02-10 17:24:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:07, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:07, INFO: Calculating and adding roadway network variable: assignable
2021-02-10 17:24:07, INFO: Calculating and adding roadway network variable: assignable
2021-02-10 17:24:20, INFO: Finished determining assignable using variable: assignable
2021-02-10 17:24:20, INFO: Finished determining assignable using variable: assignable


1.0    695049
0.0     57163
Name: assignable, dtype: int64

In [12]:
r_net = mtc.calculate_transit(
    roadway_network = r_net,
    parameters = parameters,
    network_variable = 'transit')

r_net.links_df.transit.value_counts()

2021-02-10 17:24:21, INFO: Determining transit
2021-02-10 17:24:21, INFO: Determining transit
2021-02-10 17:24:21, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:21, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:21, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:21, INFO: Lasso base directory set as: C:/Users/USGS667806/Lasso
2021-02-10 17:24:21, INFO: Calculating and adding roadway network variable: transit
2021-02-10 17:24:21, INFO: Calculating and adding roadway network variable: transit
2021-02-10 17:24:21, INFO: Finished determining transit-only variable: transit
2021-02-10 17:24:21, INFO: Finished determining transit-only variable: transit


0    1634030
1        739
Name: transit, dtype: int64

## Create a Scenario

In [13]:
base_scenario = {"road_net": r_net, "transit_net": transit_net}
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2021-02-10 17:24:22, INFO: Creating Scenario
2021-02-10 17:24:22, INFO: Creating Scenario


## Write to Disk

In [14]:
working_scenario_filename = os.path.join(output_dir, 'working_scenario_01.pickle')
pickle.dump(working_scenario, open(working_scenario_filename, 'wb'))

In [15]:
pd.crosstab(r_net.links_df.ft, r_net.links_df.roadway)

roadway,cycleway,footway,motorway,motorway_link,primary,primary_link,residential,secondary,secondary_link,service,tertiary,tertiary_link,trunk,trunk_link
ft,,,,,,,,,,,,,,
1,0,0,2774,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,3252,0
3,0,0,0,4983,0,0,0,0,0,0,0,0,0,1382
4,0,0,0,0,15505,0,0,17933,0,0,4894,0,0,0
5,0,0,0,0,8432,0,0,24996,0,0,24034,0,882,0
6,0,0,0,0,2555,1506,0,25600,1566,0,65019,611,26,0
7,0,0,0,0,0,0,538666,0,0,0,0,0,0,0
99,98614,247489,0,0,0,0,0,0,0,541983,0,0,0,0
